In [1]:
!python -V

Python 3.11.11


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

# Question 1

In [5]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
df.shape

(3066766, 19)

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Question 2

In [7]:

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculate duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Compute the standard deviation of the duration
duration_std = df['duration'].std()
print(duration_std)


42.594351241920904


# Question 3

In [8]:

filtered_data = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Compute the fraction of records left
fraction_left = len(filtered_data) / len(df)
print(fraction_left)


0.9812202822125979


# Question 4

In [9]:
from sklearn.feature_extraction import DictVectorizer

# Select the columns 
location_data = filtered_data[['PULocationID', 'DOLocationID']]

location_data = location_data.astype(str)
location_dict = location_data.to_dict(orient='records')

# Apply the dictionary vectorizer
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(location_dict)

print(f"Dimensionality of the feature matrix: {X.shape[1]}")


Dimensionality of the feature matrix: 515


# Question 5

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Train a linear regression model
model = LinearRegression()
y = filtered_data['duration']  # Response variable

# Train on the feature matrix X
model.fit(X, y)

# Predict on the training data
y_pred_train = model.predict(X)

# Calculate the RMSE (Root Mean Squared Error) on the training data
rmse_train = np.sqrt(mean_squared_error(y, y_pred_train))
print(f"RMSE on train: {rmse_train}")


la réponse : 7,64

Training on chunks because the dataset is too large

In [10]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Train a linear regression model in chunks using SGD
model = SGDRegressor(loss='squared_error', learning_rate='constant', eta0=0.0001)

y = filtered_data['duration'].values


# Function for chunks
def get_chunks(X, y, chunk_size=100000):
    for start in range(0, len(X), chunk_size):
        end = start + chunk_size
        yield X[start:end], y[start:end]

# Incremental training
for X_chunk, y_chunk in get_chunks(X ,y, chunk_size=100000):
    model.partial_fit(X_chunk, y_chunk)


y_pred_train = model.predict(X)

# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y, y_pred_train))
print(f"RMSE on train: {rmse_train}")


RMSE on train: 8.359926779483477


# Question 6

In [12]:
# Load the February 2023 dataset
val_df = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

# Repeat the preprocessing for February data (same as January)

val_df['tpep_pickup_datetime'] = pd.to_datetime(val_df['tpep_pickup_datetime'])
val_df['tpep_dropoff_datetime'] = pd.to_datetime(val_df['tpep_dropoff_datetime'])

# Calculate duration in minutes
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).dt.total_seconds() / 60


val_filtered_data = val_df[(val_df['duration'] >= 1) & (val_df['duration'] <= 60)]
val_location_data = val_filtered_data[['PULocationID', 'DOLocationID']].astype(str)
val_location_dict = val_location_data.to_dict(orient='records')
X_val = vectorizer.transform(val_location_dict)

# Predict on the validation data (February)
y_val = val_filtered_data['duration']
y_pred = model.predict(X_val)

# Calculate RMSE on the validation dataset
rmse_val= np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE on validation: {rmse_val}")


RMSE on validation: 8.347490723665683


7,81